# Building a chatbot using language models

In [1]:
import os 
from dotenv import load_dotenv
load_dotenv()
  
groq_api_key=os.getenv("GROQ_API_KEY")
groq_api_key

'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'

In [2]:
from langchain_groq import ChatGroq
model = ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)


In [3]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi , My name is Saurabh and i am a Data Scientist")])


AIMessage(content="Hello Saurabh,\n\nIt's nice to meet you! I'm a large language model, here to help with any questions or tasks you might have.\n\nWhat can I do for you today? Are you working on a particular project or just interested in chatting about data science?\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 61, 'prompt_tokens': 22, 'total_tokens': 83, 'completion_time': 0.110909091, 'prompt_time': 0.002150395, 'queue_time': 0.23116249, 'total_time': 0.113059486}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-110b6a90-5615-4060-b2f1-1d7ac00d5993-0', usage_metadata={'input_tokens': 22, 'output_tokens': 61, 'total_tokens': 83})

In [4]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi, My name is saurabh and i am a data scientist"),
        AIMessage(content="Hello Saurabh, it's nice to meet you! a large language model, I'm always interested in learning from people in data science. What kind of work do you do as a data scientist"),
        HumanMessage(content="Hey What is my name and what do i do?")
    ]
)


AIMessage(content='You said your name is Saurabh and that you are a data scientist.  😊  \n\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 84, 'total_tokens': 106, 'completion_time': 0.04, 'prompt_time': 0.005677587, 'queue_time': 0.23238837199999998, 'total_time': 0.045677587}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-6b3ffcc4-291c-4bde-989e-1de0cc8d3a3e-0', usage_metadata={'input_tokens': 84, 'output_tokens': 22, 'total_tokens': 106})

In [5]:
## Message History 
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}
def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]  

with_message_history=RunnableWithMessageHistory(model,get_session_history)


In [6]:
config={"configurable":{"session_id":"chat1"}}

In [7]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi my name is saurabh and i am a data scientist")],
        config=config
)

In [8]:
response.content

"Hi Saurabh, it's nice to meet you! \n\nThat's awesome that you're a data scientist. What kind of work do you do?  \n\nI'm always interested in learning more about how people use data to solve problems and make insights.\n"

In [9]:
response=with_message_history.invoke(
    [HumanMessage(content="what is my name")],
        config=config
)

In [10]:
response

AIMessage(content='Your name is Saurabh. 😊  I remember that from our introduction! \n\n\nIs there anything else I can help you with today?\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 92, 'total_tokens': 123, 'completion_time': 0.056363636, 'prompt_time': 0.00429595, 'queue_time': 0.236610185, 'total_time': 0.060659586}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-4cc62cba-c391-4284-8d3e-3bd0ffa65ce2-0', usage_metadata={'input_tokens': 92, 'output_tokens': 31, 'total_tokens': 123})

In [11]:
## Changing the session id --> 
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke([
    HumanMessage(content="What is my name")],
    config=config1
    )
response.content

"As an AI, I have no memory of past conversations and do not know your name. If you'd like to tell me your name, I'd be happy to use it!\n"

In [12]:
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke([
    HumanMessage(content="Hey my name is Jevin")],
    config=config1
    )
response.content


"Hi Jevin, it's nice to meet you! How can I help you today? 😊  \n\n"

# Prompt templates-->
Help to turn raw user information into format that can LLm can work with.

In [15]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt= ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Answer all the questions to the best of your ability."),
    MessagesPlaceholder(variable_name="messages")
])

chain = prompt|model

In [17]:
chain.invoke({"messages":[HumanMessage(content="Hi my name is saurabh")]})

AIMessage(content="Hi Saurabh!  It's nice to meet you.\n\nWhat can I do for you today? 😊  I'm ready to answer any questions you have or help with any tasks. \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 32, 'total_tokens': 77, 'completion_time': 0.081818182, 'prompt_time': 0.002322966, 'queue_time': 0.23217565199999998, 'total_time': 0.084141148}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-d6a19c75-6f46-4f69-9f87-05bb54aec57c-0', usage_metadata={'input_tokens': 32, 'output_tokens': 45, 'total_tokens': 77})

In [ ]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [19]:
response=with_message_history.invoke(
    [HumanMessage(content="What is my name?")],
    config=config
)
response.content


"Your name is Saurabh.  \n\nIt seems like you're checking!  Is there something specific you want to know about your name, or are you just curious?  I'm happy to chat about it further.\n"

In [46]:
## Add more complexity

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import SystemMessage

# Correct ChatPromptTemplate initialization
prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content="You are a helpful assistant. Answer all questions to the best of your ability."),
    MessagesPlaceholder(variable_name="messages")
])

chain=prompt | model

In [47]:
response=chain.invoke({"messages":[HumanMessage(content="Hi my name is saurabh")],"language":"Hindi"})
response.content

"Hi Saurabh! It's nice to meet you. 👋\n\nHow can I help you today? 😊  \n"

In [48]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    get_messages_key='messages'

)

In [ ]:
from langchain_core.messages import HumanMessage

config = {"configurable": {"session_id": "chat4"}}

response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="Hi, my name is Saurabh")]},
    config=config
)

print(response)

content="Hi Saurabh, it's nice to meet you! \n\nIs there anything I can help you with today?  😊 \n\n" additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 84, 'total_tokens': 115, 'completion_time': 0.056363636, 'prompt_time': 0.004268533, 'queue_time': 0.236771228, 'total_time': 0.060632169}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None} id='run-efd789de-44c4-4b95-8107-404c4c134e61-0' usage_metadata={'input_tokens': 84, 'output_tokens': 31, 'total_tokens': 115}


# Managing the chat conversation history-->
One important concept to understand when building chatbot is how to manage conversation histroy.if left unmanaged the list of messages will grow unbound and potentially overflow the context window of the LLm therefore it is important to add steps to limit the size of messages your are passing in .

In [52]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
    max_tokens=70,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)

messages=[
    SystemMessage(content="You are a good assistant"),
    HumanMessage(content="Hi i am bob"),
    AIMessage(content="Hi"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2+2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="No problem"),
    HumanMessage(content="having fun"),
    AIMessage(content="yes!"),

]

trimmer.invoke(messages)

c:\Users\admin\anaconda3\envs\chroma_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\admin\anaconda3\envs\chroma_env\lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\admin\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mo

[SystemMessage(content='You are a good assistant', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hi i am bob', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Hi', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2+2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='No problem', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [54]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    |prompt
    |model
)

response=chain.invoke({
    "messages":messages + [HumanMessage(content="what ice cream do i like")],
    "language":"English"}
)

response.content

'You said you like vanilla ice cream! 😊  \n'

In [55]:
## Lets wrap this in the message history
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat5"}}

In [57]:
response=chain.invoke({
    "messages":messages + [HumanMessage(content="what is my name")],
    "language":"English"}
)

response.content

'Your name is Bob.  You told me at the beginning! 😊  '